## Импорт данных

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [2]:
df = pd.read_excel('test_file.xlsx', header=1, index_col=0)

## Небольшое Data Quality

Проверка на NaN values

In [28]:
df.isna().any()

hid                      False
active_Red               False
has_Green_id             False
has_Black_id             False
has_Blue_id              False
Blue_activity_count      False
Blue_gmv_total            True
Green_gvm_total          False
Green_gvm_order_count    False
Black_gvm_total          False
Black_gvm_order_count    False
Red_android              False
Green_android            False
Black_android            False
Blue_android             False
Red_apple                False
Green_apple              False
Black_apple              False
Blue_apple               False
is_msk                   False
Red_web                  False
Green_web                False
Black_web                False
Blue_web                 False
dtype: bool

In [29]:
df[df['Blue_gmv_total'].isna()]

,hid,active_Red,has_Green_id,has_Black_id,has_Blue_id,Blue_activity_count,Blue_gmv_total,Green_gvm_total,Green_gvm_order_count,Black_gvm_total,...,Blue_android,Red_apple,Green_apple,Black_apple,Blue_apple,is_msk,Red_web,Green_web,Black_web,Blue_web
68383,18f44eb8c1abb256416294171fb63cba,0.0,1.0,0.0,3.0,4.0,NaN,820.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


Так как кейс всего один и то он относится к косячному кейсу, который я описал ниже, заменять никак NaN я не буду, строчка просто удалиться далее

Судя по документации, принадлежность пользователя к компании, городу (is_msk) и девайсу (Яблоко, АндроЕд, десктопная версия) это бинарное значение, которое может принимать 0, либо 1.
<br> Необходимо проверить так ли это на самом деле и косячные кейсы исключить

In [3]:
columns_to_check = ['active_Red', 
                    'has_Green_id', 
                    'has_Black_id', 
                    'has_Blue_id', 
                    'Red_android', 
                    'Green_android',
                    'Black_android',
                    'Blue_android',
                    'Red_apple',
                    'Green_apple',
                    'Black_apple',
                    'Blue_apple',
                    'is_msk',
                    'Red_web',
                    'Green_web',
                    'Black_web',
                    'Blue_web']

In [4]:
clean_df = df[~(df[columns_to_check] > 1).any(axis=1)]

Также в некоторых числовых по смыслу колонках в исходном датасете были замечены datetime объекты, что я также буду считать ошибочными значениями.
<br> Так как некоторые такие колонки имеют тип object я поменяю их тип на float64 как и остальные, а в тех случаях, где выдаст ошибку - исправлю

In [5]:
clean_df.dtypes

hid                       object
active_Red               float64
has_Green_id             float64
has_Black_id             float64
has_Blue_id              float64
Blue_activity_count       object
Blue_gmv_total           float64
Green_gvm_total          float64
Green_gvm_order_count     object
Black_gvm_total          float64
Black_gvm_order_count     object
Red_android              float64
Green_android            float64
Black_android            float64
Blue_android             float64
Red_apple                float64
Green_apple              float64
Black_apple              float64
Blue_apple               float64
is_msk                   float64
Red_web                  float64
Green_web                float64
Black_web                float64
Blue_web                 float64
dtype: object

In [6]:
columns_to_convert = ['Blue_activity_count', 'Green_gvm_order_count', 'Black_gvm_order_count']

In [7]:
clean_df[columns_to_convert] = clean_df[columns_to_convert].astype('float64')

C:\Users\60112813\AppData\Local\Temp\ipykernel_9964\567450683.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df[columns_to_convert] = clean_df[columns_to_convert].astype('float64')


In [8]:
clean_df.dtypes

hid                       object
active_Red               float64
has_Green_id             float64
has_Black_id             float64
has_Blue_id              float64
Blue_activity_count      float64
Blue_gmv_total           float64
Green_gvm_total          float64
Green_gvm_order_count    float64
Black_gvm_total          float64
Black_gvm_order_count    float64
Red_android              float64
Green_android            float64
Black_android            float64
Blue_android             float64
Red_apple                float64
Green_apple              float64
Black_apple              float64
Blue_apple               float64
is_msk                   float64
Red_web                  float64
Green_web                float64
Black_web                float64
Blue_web                 float64
dtype: object

Ошибок никаких не было при конвертации, все колонки имеют нужный тип данных по логике, можно переходить к EDA

## EDA Анализ

### Кол-во пользователей

In [9]:
clean_df['hid'].nunique()

99475

In [39]:
clean_df['hid'].duplicated().any()

False

Всего у нас 99 475 уникальных пользователей, дубликатных значений их ID нет

In [37]:
clean_df[clean_df[['active_Red', 'has_Green_id', 'has_Black_id', 'has_Blue_id']].sum(axis=1) == 0]['hid'].nunique()

38354

In [10]:
clean_df[clean_df[['active_Red', 'has_Green_id', 'has_Black_id', 'has_Blue_id']].sum(axis=1) == 1]['hid'].nunique()

52491

In [11]:
clean_df[clean_df[['active_Red', 'has_Green_id', 'has_Black_id', 'has_Blue_id']].sum(axis=1) == 2]['hid'].nunique()

8255

In [12]:
clean_df[clean_df[['active_Red', 'has_Green_id', 'has_Black_id', 'has_Blue_id']].sum(axis=1) == 3]['hid'].nunique()

357

In [13]:
clean_df[clean_df[['active_Red', 'has_Green_id', 'has_Black_id', 'has_Blue_id']].sum(axis=1) == 4]['hid'].nunique()

18

Из них 38 354 не имеют зарегистрированного аккаунта нигде, 52 491 имеют аккаунт только в одной компании, 8 255 - в двух компаниях, 357 - в трёх и всего 18 пользователей во всех четырех
<br> Для наглядности - график ниже. Предполагаем, что пользователь на нём может входить в несколько компаний

In [227]:
values = [
    clean_df[clean_df[['active_Red', 'has_Green_id', 'has_Black_id', 'has_Blue_id']].sum(axis=1) == 0]['hid'].nunique(),
    clean_df[clean_df[['active_Red', 'has_Green_id', 'has_Black_id', 'has_Blue_id']].sum(axis=1) == 1]['hid'].nunique(),
    clean_df[clean_df[['active_Red', 'has_Green_id', 'has_Black_id', 'has_Blue_id']].sum(axis=1) == 2]['hid'].nunique(),
    clean_df[clean_df[['active_Red', 'has_Green_id', 'has_Black_id', 'has_Blue_id']].sum(axis=1) == 3]['hid'].nunique(),
    clean_df[clean_df[['active_Red', 'has_Green_id', 'has_Black_id', 'has_Blue_id']].sum(axis=1) == 4]['hid'].nunique()
]
bin_names = ['Не зарегистрирован', 'В 1-ой компании', 'В 2-ух компаниях', 'В 3-х компаниях', 'В 4-х компаниях']

trace = go.Bar(x=bin_names, y=values, text=values)

layout = go.Layout(
    title='Кол-во зарегистрированных пользователей в компаниях',
    xaxis=dict(title='Кол-во компаний'),
    yaxis=dict(title='Кол-во пол-ей')
)

figure = go.Figure(data=trace, layout=layout)

figure.show()

Теперь можно посмотреть кол-во пользователей в каждой из компаний. 
<br> Тут также предполагаем, что пользователь может входить в несколько компаний

In [219]:
number_of_users = (clean_df[['hid','active_Red', 'has_Green_id', 'has_Black_id', 'has_Blue_id']]
 .groupby(['active_Red', 'has_Green_id', 'has_Black_id', 'has_Blue_id']).agg({'hid' : 'nunique'})
).reset_index()

In [220]:
renamed_columns = {
    'active_Red' : 'RC',
    'has_Green_id' : 'GC',
    'has_Black_id' : 'Black C',
    'has_Blue_id' : 'Blue C',
    'hid' : 'User ID'
}

In [221]:
number_of_users.rename(columns=renamed_columns, inplace=True)

In [222]:
number_of_users

,RC,GC,Black C,Blue C,User ID
0,0.0,0.0,0.0,0.0,38354
1,0.0,0.0,0.0,1.0,4819
2,0.0,0.0,1.0,0.0,331
3,0.0,0.0,1.0,1.0,29
4,0.0,1.0,0.0,0.0,390
5,0.0,1.0,0.0,1.0,40
6,0.0,1.0,1.0,0.0,50
7,0.0,1.0,1.0,1.0,3
8,1.0,0.0,0.0,0.0,46951
9,1.0,0.0,0.0,1.0,7528


In [223]:
number_of_users['Company'] = (
    number_of_users.apply(lambda row: ', '.join([col for col in number_of_users.columns if row[col] == 1]), axis=1)
)

In [224]:
number_of_users['Company'][0] = 'No Company'

C:\Users\60112813\AppData\Local\Temp\ipykernel_9964\4278020537.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [228]:
trace = go.Bar(x=number_of_users['Company'], y=number_of_users['User ID'])

layout = go.Layout(
    title='Кол-во зарегистрированных пользователей в компаниях',
    xaxis=dict(title='Принадлежность к компании'),
    yaxis=dict(title='Кол-во пол-ей')
)

figure = go.Figure(data=[trace], layout=layout)

figure.show()

В целом, что по сводной таблички, что по графику видно, что абсолютные лидеры по кол-ву пользователей это пользователи незарегистрированные нигде и пользователи из Red company.
<br> Далее идут пользователи, которые одновременно принадлежат и к Red company, и к Blue company, и пользователи, которые имееют аккаунт только в Blue company. (Для удобства отображение "Company" Было сокращено до "C")
<br> К сожалению, кол-во уникальных пользователей в каждой из компании не представляется возможным найти, так как нет даты регистрации в каждой из компаний, поэтому непонятно, в каком из сервисов пользователь зарегистрировался первее всего

### Распределение кол-ва успешных заказов в сервисе

К сожалению, датасет неполный и можно оценить кол-во успешных заказов и их доли только у компаний Green и Black
<br> Для большей точности я оставляю только тех пользователей, которые зарегистрированы хотя бы в одном из этих сервисов

In [281]:
orders_df = clean_df[['hid', 'Green_gvm_order_count', 'Black_gvm_order_count', 'has_Green_id', 'has_Black_id']]

In [282]:
orders_df = orders_df[(orders_df['Green_gvm_order_count'] + orders_df['Black_gvm_order_count'] > 0)]

In [287]:
orders_df[orders_df['has_Green_id'] > 0]['Green_gvm_order_count'].describe()

count    874.000000
mean       8.756293
std       15.872743
min        1.000000
25%        1.000000
50%        4.000000
75%        9.000000
max      168.000000
Name: Green_gvm_order_count, dtype: float64

In [288]:
orders_df[orders_df['has_Black_id'] > 0]['Black_gvm_order_count'].describe()

count    1065.000000
mean        4.051643
std         5.971751
min         1.000000
25%         1.000000
50%         2.000000
75%         4.000000
max        60.000000
Name: Black_gvm_order_count, dtype: float64

Всего успешных заказов у компании Green 7 653, максимальный заказ - 168 абстрактных штук, а у компании Black 4 315, 60 абстрактных штук
<br> Так как минимальное кол-во заказов в обоих компаниях равно одному, можно сказать, что в обоих компаниях каждый зарегистрированный пользователь заказывал хотя бы один раз, так что на долю успешных заказов в принципе в рамках компании смотреть бесполезно, она равно 100 %

### Распределение активных действий пол-тя в сервисе

К сожалению, опять же из-за неполноты датасеты, количество активных действий есть только у одной компании - Blue

In [306]:
activity_counts = clean_df[['hid', 'Blue_activity_count', 'has_Blue_id']]

In [308]:
activity_counts = activity_counts[activity_counts['has_Blue_id'] == 1]

In [310]:
activity_counts['Blue_activity_count'].describe()

count    12746.000000
mean         1.493802
std          5.484670
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max        237.000000
Name: Blue_activity_count, dtype: float64

Видим, что максимальное кол-во активных действий у пользователя 237, а минимальное 0. Тут долю активных/неактивных пол-ей посмотреть можно
<br> <b>P.S.</b> Также брал только тех пользователей, которые зареганы в Blue

In [316]:
(activity_counts[activity_counts['Blue_activity_count'] > 0]['hid'].nunique() / activity_counts['hid'].nunique()) * 100

27.875411893927506

Процентная доля активных пользователей составляет почти 28 % от общего кол-ва пользователей, зарегистрированных в сервисе
<br> Как будто сервис не очень популярный, если только около трети пользователей как-то в нем активничают

### Распределение GVM

Опять же, тут я оцениваю компании, если пользователь в ней зарегистрирован

In [319]:
gvm_total = (clean_df[[
    'hid', 'Blue_gmv_total', 
    'Green_gvm_total', 'Black_gvm_total', 
    'Blue_activity_count', 'Green_gvm_order_count', 
    'Black_gvm_order_count', 'has_Blue_id',
    'has_Black_id', 'has_Green_id'
    ]]
)

In [324]:
gvm_total_blue = gvm_total[gvm_total['has_Blue_id'] == 1]

In [336]:
gvm_total_blue.sum()

hid                      00003c7fa5e57ce672f1801f2679f5c40001aede5ec5c9...
Blue_gmv_total                                                    233704.0
Green_gvm_total                                                   874218.0
Black_gvm_total                                                  1990816.0
Blue_activity_count                                                19040.0
Green_gvm_order_count                                               1159.0
Black_gvm_order_count                                                989.0
has_Blue_id                                                        12746.0
has_Black_id                                                         265.0
has_Green_id                                                         155.0
dtype: object

Пользователи, зарегистрированные в компании Blue приносят тотальный GVM 233 704
<br> Однако, также они могут быть зарегистрированы в компаниях Black или Green или (возможно) в обоих.
<br> Компании Green они приносят 874 218 GVM, а компании Black больше всего - 1 990 816 GVM

In [345]:
gvm_total_green = gvm_total[gvm_total['has_Green_id'] == 1]

In [346]:
gvm_total_green.sum()

hid                      00096dc15d53c959cbff731eba795dfb00272111ca0bf4...
Blue_gmv_total                                                     15536.0
Green_gvm_total                                                  5935926.0
Black_gvm_total                                                  1556360.0
Blue_activity_count                                                  371.0
Green_gvm_order_count                                               7653.0
Black_gvm_order_count                                                698.0
has_Blue_id                                                          155.0
has_Black_id                                                         116.0
has_Green_id                                                         874.0
dtype: object

Пользователи зарегистрированные в компании Green приносят ей 5 935 926 GVM,
<br> Однако, они также могут быть зарегистрированы в двух других компаниях и приносят Blue 15 536 GVM и Black 1 556 360 GVM

In [347]:
gvm_total_black = gvm_total[gvm_total['has_Black_id'] == 1]

In [348]:
gvm_total_black.sum()

hid                      001c3f004c05017d60fa91f00cdbcdda003017284d22a5...
Blue_gmv_total                                                      6350.0
Green_gvm_total                                                  1187054.0
Black_gvm_total                                                  9354698.0
Blue_activity_count                                                  689.0
Green_gvm_order_count                                               1417.0
Black_gvm_order_count                                               4315.0
has_Blue_id                                                          265.0
has_Black_id                                                        1065.0
has_Green_id                                                         116.0
dtype: object

Пользователи, зарегистрированные в компании Black приносят ей 9 354 698 GVM,
Однако, ситуация такая же как и выше, поэтому Blue они также приносят 6 350 GVM, а Green 1 187 054 GVM

### Распределение геолокации и девайсов для использования

In [298]:
(clean_df[clean_df['is_msk'] == 1]['hid'].nunique() / clean_df['hid'].nunique()) * 100

11.893440562955517

Видим, что пользователей из Москвы всего около 12 %

In [371]:
clean_df[(clean_df[['Red_apple', 'Green_apple', 'Black_apple', 'Blue_apple']] > 0).any(axis=1)]['hid'].nunique()

10959

In [372]:
clean_df[(clean_df[['Red_android', 'Green_android', 'Black_android', 'Blue_android']] > 0).any(axis=1)]['hid'].nunique()

33073

In [373]:
clean_df[(clean_df[['Red_web', 'Green_web', 'Black_web', 'Blue_web']] > 0).any(axis=1)]['hid'].nunique()

67164

Распределение видим следующие: с яблоком 10 959 пользователей, с андроЕдом - 33 073, а с веб версией 67 164
<br> Опять же, не забываем, что у нескольких пользователей может быть несколько девайсов.

#### Разбиение пользователей на группы

In [301]:
red_users = clean_df[clean_df['active_Red'] == 1]

In [357]:
green_users = clean_df[clean_df['has_Green_id'] == 1]

In [358]:
blue_users = clean_df[clean_df['has_Blue_id'] == 1]

In [363]:
black_users = clean_df[clean_df['has_Black_id'] == 1]

#### Пользователи Red компании

In [376]:
red_users[red_users['Red_apple'] == 1]['hid'].nunique()

9013

In [379]:
(red_users[red_users['Red_apple'] == 1]['hid'].nunique() / red_users['hid'].nunique()) * 100

16.251645359635045

In [380]:
red_users[red_users['Red_android'] == 1]['hid'].nunique()

24855

In [382]:
(red_users[red_users['Red_android'] == 1]['hid'].nunique() / red_users['hid'].nunique()) * 100

44.81689175787519

In [383]:
red_users[red_users['Red_web'] == 1]['hid'].nunique()

45196

In [384]:
(red_users[red_users['Red_web'] == 1]['hid'].nunique() / red_users['hid'].nunique()) * 100

81.49443733208315

У пользователей Red компании разбиение видим следующее:
1. Эпл в меньшинстве, всего 9 013, 16 % общего кол-ва пользователей
2. АндроЕд наступает, 24 855, это почти 45 %
3. Веб-версия выигрывает, 45 196 человек, 81 %
<br> <b> P.S. </b> Да, процентов суммарно получается больше 100, так как один и тот же пользователь может пользоваться сервисом и с мобильного девайса (а может даже с двух, если богач), и с веб версии. Однако все равно можно сделать вывод, что веб версией пользуются больше всего

#### Пользователи Green компании

In [400]:
green_users[green_users['Green_apple'] == 1]['hid'].nunique()

226

In [399]:
(green_users[green_users['Green_apple'] == 1]['hid'].nunique() / green_users['hid'].nunique()) * 100

25.85812356979405

In [389]:
green_users[green_users['Green_android'] == 1]['hid'].nunique()

179

In [401]:
(green_users[green_users['Green_android'] == 1]['hid'].nunique() / green_users['hid'].nunique()) * 100

20.48054919908467

In [404]:
green_users[green_users['Green_web'] == 1]['hid'].nunique()

26

In [402]:
(green_users[green_users['Green_web'] == 1]['hid'].nunique() / green_users['hid'].nunique()) * 100

2.9748283752860414

In [393]:
green_users['hid'].nunique()

874

In [405]:
green_users[(green_users['Green_android'] == 0) & (green_users['Green_apple'] == 0) & (green_users['Green_web'] == 0)]['hid'].nunique()

470

In [407]:
(green_users[(green_users['Green_android'] == 0) & (green_users['Green_apple'] == 0) & (green_users['Green_web'] == 0)]['hid'].nunique() / green_users['hid'].nunique()) * 100

53.775743707093824

У зеленых ситуация следующая:
1. На удивление больше всего яблочников, 226 человек, это 25 %
2. АндроЕдов в этом случае поменьше, 179 человек, это 20 %
3. Веб версией почти никто не пользуется, всего 26 человек, это почти 3 %
4. Однако, есть большая прослойка людей, которая приносит GVM, но также не пользуется ни одним из девайсов, это почти 54 %, 470 человек. Так что делаю предположение, что возможно это какой-то в большей степени оффлайн сервис